<h1><center>Generating a Population of Non-Lensed Galaxies</center></h1>

This notebook is used to generate images of non-lensed galaxies. These images
can be used to train the model.

This notebook is based on 
Narayan Khadka's [`SLSim`](https://github.com/LSST-strong-lensing/slsim).

<h3><center>Table of Contents</center></h3>

1. [Libraries & Dependencies](#libraries_dependencies)
2. [Setting the Universe's Parameters](#setting_parameters)
3. [Generating a Population of Galaxies](#generating_population)
   1. [Set Requirements for Simulation](#generating_population1)
   2. [Set Up Simulation Pipeline](#generating_population2)
   3. [Initiate False Positive Population Class](#generating_population3)
   4. [Draw Non-Lenses](#generating_population4)
4. [Simulate Images](#simulate_images)
   1. [Simulate Images in the <i>i</i>, <i>r</i>, and <i>g</i> Bands](#simulate_images1)
   2. [Simulate Images in the RGB Color Scale](#simulate_images2)
5. [Vizualize the Simulated Images](#visualize)
6. [Save Individual Images](#save)
7. [<i>Additional: Delete Images</i>](#delete)

<a id='libraries_dependencies'></a>  <!-- Cell ID -->
<h2>1. Libraries & Dependencies</h2>

In [4]:
# From astropy
from astropy.cosmology import FlatLambdaCDM
from astropy.units     import Quantity

# From slsim
from slsim import Pipelines, Deflectors, Sources
from slsim.FalsePositives.false_positive_pop import FalsePositivePop
from slsim.image_simulation import simulate_image
from slsim.image_simulation import rgb_image_from_image_list
from slsim.Plots.plot_functions import create_image_montage_from_image_list

# Other libraries
from tqdm import tqdm
import numpy as np
import os, imageio # To save images
import shutil # To empty directory

<a id='setting_parameters'></a>  <!-- Cell ID -->
<h2>2. Setting the Universe's Parameters</h2>

First, we pick a cosmological model. This tells `astropy` to assume a flat
universe with a Hubble constant of $H_0 = 70 \text{~km s}^{-1}$
$\text{Mpc}^{-1}$ and a matter density parameter $\Omega_\text{m} = 0.3$.

We’ll use this `cosmo` object whenever we need to convert between redshift 
and distance or compute comoving volumes.

In [5]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

Next, we define how big a piece of sky we’re simulating versus the full survey footprint.

In [6]:
sky_area      = Quantity(value=2,   unit="deg2") #   2 deg²
sky_area_full = Quantity(value=230, unit="deg2") # 230 deg²

<a id='generating_population'></a>  <!-- Cell ID -->
<h2>3. Generating a Population of Galaxies</h2>

<a id='generating_population1'></a>  <!-- Cell ID -->
<h3>3.1. Set Requirements for Simulation</h3>

To start, we specify simple "cuts" on our two galaxy populations (deflectors
and background sources) so that we only draw objects that are bright enough
and lie in the redshift ranges we care about.

In [7]:
kwargs_deflector_cut = {
    "band": "i",
    "band_max": 27,
    "z_min": 0.01,
    "z_max": 1.5
}

kwargs_source_cut = {
    "band": "i",
    "band_max": 25,
    "z_min": 0.01,
    "z_max": 5.0
}

Together, these dictionaries tell our sampling routine to select deflector
galaxies no fainter than magnitude $m_i = 27$ in the <i>i</i>-band between
$z = 0.01$ to $1.5$, and source galaxies no fainter than $m_i = 25$ between
$z = 0.01$ to $5.0$. That way, we generate only the objects that will appear
(and potentially lens) in our simulated images.

<a id='generating_population2'></a>  <!-- Cell ID -->
<h3>3.2. Set Up Simulation Pipeline</h3>

Here, we instantiate `skypy`'s "pipeline" object.

By passing in our survey area (`sky_area`) and cosmology (`cosmo`), it knows
where and how big a patch of sky to simulate.

In [8]:
simulation_pipeline = Pipelines.SkyPyPipeline(
    skypy_config = None,
    sky_area = sky_area,
    filters = None,
    cosmo = cosmo
)

Then, we draw a catalog of red (early-type) galaxies. The `red_galaxies`
property runs through the pipeline’s galaxy-population recipe and returns an
array of simulated red galaxies.

In [9]:
deflector_galaxy_list = simulation_pipeline.red_galaxies

Now, we turn our raw catalog of red galaxies into a fully-specified set of lens
deflectors that `slsim` can use to make images.

In [10]:
deflector_galaxies = Deflectors.EllipticalLensGalaxies(
    # Filter only brightest systems
    galaxy_list = deflector_galaxy_list[deflector_galaxy_list["mag_i"] > 18],
    # Pass the deflector requirements
    kwargs_cut = kwargs_deflector_cut,
    # Default mass-to-light prescription
    kwargs_mass2light = None,
    cosmo = cosmo,
    sky_area = sky_area
)

/Users/nataliaraymundipinheiro/Documents/Stony Brook University/Semesters/Spring 2025/PHY688/projects/lensnet/external/slsim/slsim/Deflectors/elliptical_lens_galaxies.py:46: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  galaxy_list = param_util.catalog_with_angular_size_in_arcsec(


For the source galaxies, we take the blue (late-type) galaxies.

In [11]:
source_galaxy_list = simulation_pipeline.blue_galaxies

The next step builds our background source catalog.

In [12]:
source_galaxies = Sources.Galaxies(
    galaxy_list = source_galaxy_list,
    # Pass the source requirements
    kwargs_cut = kwargs_source_cut,
    cosmo = cosmo,
    sky_area = sky_area,
    catalog_type = "skypy",
    # Default skypy prescription
    downsample_to_dc2 = False,
)

<a id='generating_population3'></a>  <!-- Cell ID -->
<h3>3.3. Initiate False Positive Population Class</h3>

We take the deflector and source catalogs and wrap them into a "galaxy–galaxy"
population object that we’ll hand off to the imaging code. `population_generator`
will hold all of the pairings we need to loop over when we render our images.

In [13]:
population_generator = FalsePositivePop(
    deflector_galaxies,
    source_galaxies,
    cosmo = cosmo,
    # Simulate exactly our defined sky_area
    test_area_factor = 1,
    # Randomly assign either one, two, or three sources behind each lens
    source_number_choice = [1, 2, 3]
)

<a id='generating_population4'></a>  <!-- Cell ID -->
<h3>3.4. Draw Non-Lenses</h3>

Tell `FalsePositivePop` to pull a batch of false-positives (non-lensed galaxy pairs) from
the galaxy-galaxy population. The chosen number is the quantity of specific
lens-source pairings that produce no strong-lensing signature.

In [14]:
# If number is too large, there may be a memory issue
false_positive_population = population_generator.draw_false_positive(
    number = 100
)

`population_generator` is the recipe or blueprint for pairing lenses and
sources. `false_positive_population` is the actual list of a certain number of
paired instances drawn from that recipe, all of which are "negative"; they are
ready to be rendered into images and used in the learning model as negative
examples.

<a id='simulate_images'></a>  <!-- Cell ID -->
<h2>4. Simulate Images</h2>


Now that we have the information to render the images, we can start creating
them so we can feed them into our learning model.

<a id='simulate_images1'></a>  <!-- Cell ID -->
<h3>4.1. Simulate Images in the <i>i</i>, <i>r</i>, and <i>g</i> Bands</h3>

We’re taking each of your non-lensed galaxy–galaxy pairings and turning them
into three separate mock observations: one in the <i>i</i>-band, one in the
<i>r</i>-band, and one in the <i>g</i>-band.

In [15]:
image_list_i, image_list_r, image_list_g = [], [], []

# Parameters to simulate image
# All images use the same parameters
num_pix = 128 # Number of pixels per axis
add_noise = True
observatory = "LSST"
kwargs_psf = None
kwargs_numerics = None


for false_positive_pair in tqdm(false_positive_population):
    
    # i-band
    image_i = simulate_image(
        false_positive_pair,
        band = "i",
        num_pix = num_pix,
        add_noise = add_noise,
        observatory = observatory,
        kwargs_psf = kwargs_psf,
        kwargs_numerics = kwargs_numerics
    )
    image_list_i.append(image_i)
    
    # r-band
    image_r = simulate_image(
        false_positive_pair,
        band = "r",
        num_pix = num_pix,
        add_noise = add_noise,
        observatory = observatory,
        kwargs_psf = kwargs_psf,
        kwargs_numerics = kwargs_numerics
    )
    image_list_r.append(image_r)
    
    # g-band
    image_g = simulate_image(
        false_positive_pair,
        band = "g",
        num_pix = num_pix,
        add_noise = add_noise,
        observatory = observatory,
        kwargs_psf = kwargs_psf,
        kwargs_numerics = kwargs_numerics
    )
    image_list_g.append(image_g)
    

  0%|                                                                                                                                                      | 0/100 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.12/site-packages/lenstronomy/LensModel/lens_model.py:96: UserWarning: Astropy Cosmology is provided. Make sure your cosmology model is consistent with the cosmology_model argument.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 32.31it/s]


<a id='simulate_images2'></a>  <!-- Cell ID -->
<h3>4.2. Simulate Images in the RGB Color Scale</h3>

Simulating RGB (or, more generally, multi-band) cutouts gives the model other
types of information.

In [16]:
image_list_rgb = []

for image_i in tqdm(range(len(image_list_i))):
    image_rgb = rgb_image_from_image_list(
        [
            image_list_i[image_i],
            image_list_r[image_i],
            image_list_g[image_i]
        ],
        # Controls intensity scaling so that faint arcs are visible
        stretch = 0.5
    )
    image_list_rgb.append(image_rgb)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1061.46it/s]


<a id='visualize'></a>  <!-- Cell ID -->
<h2>5. Vizualize the Simulated Images</h2>

Grab a subset of the RGB cutouts and lay them out in a big tiled image. This
allows us to eyeball lots of examples at once.

Uncomment this code if you wish to visualize the images. Be aware that it may
take a while if the number of pixels is large.

In [17]:
# num_images = np.arange(1000)
# selected_images = []

# for i in tqdm(num_images):
#     selected_images.append(image_list_rgb[i])
    
# grid = create_image_montage_from_image_list(
#     num_rows = 84,
#     num_cols = 12,
#     images = selected_images,
#     time = None,
#     image_type = "other"
# )

<a id='save'></a>  <!-- Cell ID -->
<h2>6. Save Individual Images</h2>

In [18]:
output_dir = "../../data/raw/negative"

In [19]:
for i, image in tqdm(enumerate(image_list_rgb)):
    # i is the index of the image
    # image has shape (41, 41, 3)
    
    filename = os.path.join(output_dir, f"negative_image_{i:04d}.png")
    imageio.imwrite(filename, image)

100it [00:00, 205.56it/s]


<a id='delete'></a>  <!-- Cell ID -->
<h2>7. <i>Additional: Delete Images</i></h2>

This deletes everything that is inside `raw-negative`. Uncomment if the files
need deleting.

In [35]:
# for entry in tqdm(os.listdir(output_dir)):
#     entry_path = os.path.join(output_dir, entry)
    
#     try:
#         if os.path.isfile(entry_path) or os.path.islink(entry_path):
#             os.remove(entry_path)
#         elif os.path.isdir(entry_path):
#             shutil.rmtree(entry_path)
#     except Exception as e:
#         print(f"Failed to delete {entry_path}: {e}")